# PyTorch Internals and Autograd

### Tensor Implementation Details

In [6]:
import torch

# Create a tensor
x = torch.arange(12, dtype=torch.float32)
print(f"Original tensor x: {x}")

# Storage is a 1D array of 12 floats
print(f"Storage elements: {x.storage().tolist()}")
print(f"Storage type: {x.storage().dtype}")
print(f"Storage size: {len(x.storage())}")



Original tensor x: tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])
Storage elements: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]
Storage type: torch.float32
Storage size: 12


In [7]:
# Storage is a 1D array of 12 floats
print(f"Storage elements: {x.reshape(-1).tolist()}")
print(f"Number of elements: {x.numel()}")
print(f"Tensor dtype: {x.dtype}")

Storage elements: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]
Number of elements: 12
Tensor dtype: torch.float32


In [8]:
# Create a view by reshaping
y = x.view(3, 4)
print(f"\nReshaped tensor y:\n{y}")

# y has different shape/strides but shares the same storage
print(f"Does y share storage with x? {y.storage().data_ptr() == x.storage().data_ptr()}")

# Modifying the view affects the original (and vice versa)
y[0, 0] = 99.0
print(f"\nModified y:\n{y}")
print(f"Original x after modifying y: {x}")


Reshaped tensor y:
tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
Does y share storage with x? True

Modified y:
tensor([[99.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
Original x after modifying y: tensor([99.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])


In [3]:
# Tensor Metadata
import torch

t = torch.arange(12, dtype=torch.float32).view(3, 4) # view() requires the tensor to be contiguous
print(f"Tensor t:\n{t}")
print(f"Shape: {t.shape}")
print(f"Stride: {t.stride()}") 

# Stride: It's a tuple where the i-th element specifies the jump in memory 
# (number of elements in the storage) needed to move one step along the i-th dimension of the tensor.

Tensor t:
tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
Shape: torch.Size([3, 4])
Stride: (4, 1)


The stride (4, 1) means:

* To move one step along dimension 0 (down a row), you need to jump 4 elements in the underlying 1D `Storage`. (e.g., from element 0 to element 4).
* To move one step along dimension 1 (across a column), you need to jump 1 element in the underlying 1D `Storage`. (e.g., from element 0 to element 1).

The stride determines how the multi-dimensional tensor maps onto the linear `Storage`.

Having some better understanding of `Stride`:

`Stride: (4, 1)` means:
* Moving 1 step in dim 0 (row direction) --> jump 4 elements in memory
* Moving 1 step in dim 1 (column direction) --> jump 1 element in memory

Why 4?
because each row has 4 columns --> to go from row0 --> row1 --> skip 4 elements

Why 1?
because elements in the same row are adjacent in memory.

Memory in 1D: [ a, b, c, d, e, f, g, h, i, j, k, l]
tensor shape: 3 rows x 4 columns

Stride `(4, 1)` means:
* row step = +4 in memory
* col step = +1 in memory

so,
* x[0][0] = a
* x[1][0] = a + 4 steps = e
* x[0][1] = a + 1 step = b

In [ ]:
print(f"Is t contiguous? {t.is_contiguous()}")